In [ ]:
%pylab inline

# sourmash algorithms and implementation

topics:
    
* modulo hash, "density hashing",
    * https://github.com/dib-lab/sourmash/issues/823
    * https://github.com/dib-lab/sourmash/issues/606
    * https://github.com/richarddurbin/modimizer
* 'scaled' implementation, and using scaled hashes for Jaccard similarity and containment calculations


In [7]:
import screed

def kmerize_seq(sequence, k=31):
    sequence = sequence.upper()
    for start in range(0, len(sequence) - k + 1):
        kmer = sequence[start:start+k]
        
        # canonicalize the k-mer
        revcomp = screed.rc(kmer)
        if kmer < revcomp:
            yield kmer
        else:
            yield revcomp
            
#list(kmerize('ATGGACAGAGATG', k=2))

def kmerize_file(filename, k=31):
    # walk through every record in a FASTA/FASTQ file
    for record in screed.open(filename):
        # k-merize each sequence
        for kmer in kmerize_seq(record.sequence[:50000]):
            # return canonical version of each k-mer
            yield kmer

len(set(kmerize_file('data/genomes/2.fa')))



49970

In [8]:
import mmh3

In [25]:
import mmh3

# implementation of a minhash bottom sketch, where a fixed number
# of hashes is kept for each input set of k-mers.

# there's a bug in this code where we are not handling duplicate k-mers
# properly.
def minhash(kmers, num=500):
    basket = []
    for kmer in kmers:
        hashval = mmh3.hash64(kmer, seed=42)[0]
        if hashval < 0:
            hashval += 2**64
        
        # basket not full? add hash value
        if len(basket) < num:
            basket.append(hashval)
            basket = list(sorted(basket))
        # basket full?
        elif len(basket) == num:
            if basket[-1] > hashval:
                # this hashvalue does not belong in basket
                pass
            else:
                # is new hash value less than largest? if so, evict.
                basket.pop()
                basket.append(hashval)
                basket = list(sorted(basket))
                
    return basket

b_all = set(kmerize_file('data/genomes/2.fa'))
c_all = set(kmerize_file('data/genomes/47.fa'))
d_all = set(kmerize_file('data/genomes/63.fa'))


In [29]:
b = minhash(b_all, num=1000)
c = minhash(c_all, num=1000)
d = minhash(d_all, num=1000)


In [30]:
print(len(b_all), len(c_all), len(d_all))
print(len(b), len(c), len(d))

49970 96808 169296
1000 1000 1000


In [31]:
print(type(b_all))
print(type(b))

<class 'set'>
<class 'list'>


In [32]:
def jaccard_similarity(x, y):
    x = set(x)
    intersection = x.intersection(y)
    union = x.union(y)
    return len(intersection) / len(union)

print('b_all to b_all', jaccard_similarity(b_all, b_all))
print('b to b', jaccard_similarity(b, b))

print('b_all to c_all', jaccard_similarity(b_all, c_all))
print('b to c', jaccard_similarity(b, c))

print('c_all to d_all', jaccard_similarity(c_all, d_all))
print('c to d', jaccard_similarity(c, d))

b_all to b_all 1.0
b to b 1.0
b_all to c_all 0.0
b to c 0.0
c_all to d_all 0.2544146624303506
c to d 0.1607661056297156


In [33]:
# now instead of a minhash, implement a scaled sketch, which is what
# we recommend using in sourmash.

# scaled here is 1/f of k-mers that we are going to keep
# this is a sampling mechanism where we "stochastically" sample
# by randomizing the order of k-mers with a hash function, and
# then choosing a subset of them deterministically.

def scaledhash(kmers, scaled=1000):
    basket = set()
    
    MAX_HASH=2**64 - 1
    boundary = MAX_HASH / scaled
    
    for kmer in kmers:
        hashval = mmh3.hash64(kmer, seed=42)[0]
        if hashval < 0:
            hashval += 2**64
            
        # accept any k-mer that hashes into the bottom 1/scaled of the
        # hash space. what this should do (for a good hash function)
        # is pick approximately 1 in 'scaled' of the input k-mers.
        if hashval < boundary:
            basket.add(hashval)

    return basket
        
b_sh = scaledhash(b_all, scaled=1000)
c_sh = scaledhash(c_all, scaled=1000)
d_sh = scaledhash(d_all, scaled=1000)


In [34]:
print('b_all to b_all', jaccard_similarity(b_all, b_all))
print('b to b', jaccard_similarity(b, b))
print('b_sh to b_sh', jaccard_similarity(b_sh, b_sh))


print('b_all to c_all', jaccard_similarity(b_all, c_all))
print('b to c', jaccard_similarity(b, c))
print('b_sh to c_sh', jaccard_similarity(b_sh, c_sh))

print('c_all to d_all', jaccard_similarity(c_all, d_all))
print('c to d', jaccard_similarity(c, d))
print('c_sh to dsh', jaccard_similarity(c_sh, d_sh))

b_all to b_all 1.0
b to b 1.0
b_sh to b_sh 1.0
b_all to c_all 0.0
b to c 0.0
b_sh to c_sh 0.0
c_all to d_all 0.2544146624303506
c to d 0.1607661056297156
c_sh to dsh 0.1889400921658986


In [ ]:
topics:
    
* modulo hash, "density hashing",
    * https://github.com/dib-lab/sourmash/issues/823
    * https://github.com/dib-lab/sourmash/issues/606
    * https://github.com/richarddurbin/modimizer
* 'scaled' implementation, and using scaled hashes for Jaccard similarity and containment calculations
    * with a scaled of 10,000, you get approximately 1 hash per every 10kb of sequence
    * see [graph at 10kb](https://github.com/dib-lab/charcoal/blob/master/stats/stats10k.png), [graph at 5kb](https://github.com/dib-lab/charcoal/blob/master/stats/stats5k.png)
    * (similar statistics apply on a stream of distinct k-mers)
* looking in more depth at the signatures
* manipulating signatures (command line, Python, etc.)
* downsampling in particular
* ...hash matches do indeed correspond to nucleotide alignments!
* downsides of scaled approach:
    * don't work for small genomes
    * arbitrary growth in size (so may be unnecessarily large for large genomes!)
    
tl;dr sourmash is basically many thousands of lines of code around doing this stuff with reasonable efficiency and care.